# Scraping web data for use with a Recurrent Neural Network

In [14]:
import os
os.getcwd()+'/stuff'

'/Users/MF/Desktop/data-science-side-projects/data_scraping_project'

Example using scrapy module for data scraping, see: https://doc.scrapy.org/en/1.4/intro/overview.html

In [17]:
with open('quotes_spider.py', 'w') as swf:
    swf.write('''\
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/tag/humor/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.xpath('span/small/text()').extract_first(),
            }

        next_page = response.css('li.next a::attr("href")').extract_first()
        if next_page is not None:
            yield response.follow(next_page, self.parse)
            ''')
    swf.close()

In [20]:
! scrapy runspider quotes_spider.py -o quotes.json &>/dev/null

http://adequateman.deadspin.com/whats-the-best-store-to-daydream-about-robbing-1796232076
http://deadspin.com/tag/funbag

IDEA 1: maybe have the scrapyspider collect the urls, and then run through beautiful soup like below and write each one to its own text file. Perhaps this is formally two processes, web crawling with spider, and then html parsing and cleanup for each page

there's also some type of querying language with an x in it to use... XPATH

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

IDEA 2: For the projects... one version reproduces whole column. another version you enter a question and it generates an answer for you... would be interesting to do for an "agony aunt" column... should be a lot of data to scrape there...

TO DO: 
- Fix up formatting so apostrophes don't get messed up in html (.prettify(formatter="html")) !!!!!
- Figure out how to not repeat the contents of blockquote
- Crawl the site to make a file of all the urls. Print that to text file, so it can be loaded in and looped into bs4 stuff below

IDEA 3: scrapy might be good for formatting Q and A json files

## Scrape the URLS for each html page

Create spider and run it from shell. Porbably a nicer way to do this.

In [96]:
import scrapy

#top_url = 'http://deadspin.com/tag/funbag'

with open('funbag_spider.py', 'w') as swf:
    swf.write('''\
import scrapy

class FunbagSpider(scrapy.Spider):
    name = 'funbag_urls'
    start_urls = ['http://deadspin.com/tag/funbag']

    def parse(self, response):
        for entry in response.css('h1.entry-title'):
            yield {
                'url': entry.css('a::attr(href)').extract_first(),
            }

        next_page = response.css('div.load-more__button a::attr(href)').extract_first()
        if next_page is not None:
            next_page_url = 'http://deadspin.com/tag/funbag' + next_page
            yield scrapy.Request(next_page_url)
            ''')

In [98]:
! scrapy runspider funbag_spider.py -o funbag_urls.json &>/dev/null 

Load urls back in from json.

## Parse, sanitize, and save each of the html pages

In [43]:
import os
from bs4 import BeautifulSoup
import urllib2

my_url = 'http://adequateman.deadspin.com/whats-the-best-store-to-daydream-about-robbing-1796232076'
my_outfile = 'first.html'

def oneFullPageParse(url, outfile):
    fo = open(outfile, 'w')
    funbag = urllib2.urlopen(url)
    soup = BeautifulSoup(funbag,'html5lib') #html.parser')
    for anchor in soup.find_all(['p', 'blockquote'], class_ = ''): 
        if anchor.parent.name == 'blockquote':
            continue
        fo.write(anchor.prettify(formatter="html")) #anchor.prettify() #anchor.contents
    fo.close()
    
oneFullPageParse(my_url, my_outfile)


In [37]:
#Junked code

#from HTMLParser import HTMLParser
#parser = HTMLParser()
#parser.feed('https://docs.python.org/2/library/htmlparser.html')

#------
#%lsmagic
#!ps -a
##%time
##2+2
#------
    #'p', class_ = '' #removes the p tags that have a class attribute!!! just need blockquotes
    #['p', 'blockquote'] # almost right, but repeats the contents of the blockquote, maybe can do something with parent/child
    #'p' and 'blockquote' #gets the two used together... could just generate questions with this for a start
    #'p' or 'blockquote' #doesn't print out the blockquote tag but has the data...
    #print anchor.parent #do something with this, it works! find parent though?!
    
    #print anchor.previousSibling
    ###print soup.find_parent('anchor')
    
    #maybe have something like if parent of p is block quote, do nothing <<<<<<<<<<<<
    
#looks like we want to strip out certain classes for p, like the ad label. maybe do with text processing outside bs4
#do that, and throw in the blockquotes, and bob's your uncle?